In [1]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

We're trying to make a searchable map with one static layer and 3 toggle layers. 

Static layer = air quality monitors (both regulatory and PurpleAir), styled by 3-yr avg. PM2.5 reading

Toggle layers:
- life expectancy by tract
- asthma by tract
- POC or poverty by tract

Steps:
- pull 2020 and 2010 tracts shapefile
- restrict to just Houston metro
- pull asthma data from [CDC PLACES](https://data.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-Census-Tract-D/cwsq-ngmh/about_data)
- pull most recent POC/poverty from Census API
- pull life expectancy [from CDC](https://data.cdc.gov/NCHS/U-S-Life-Expectancy-at-Birth-by-State-and-Census-T/5h56-n989/about_data) - note, this uses the 2010 tracts
- join data files to shapes and export
- simplify geojson files in mapshaper by 20%

In [2]:
########################################
#load geos
########################################

houmetro_cntys = {'Fort Bend, TX':'48157','San Jacinto, TX':'48407','Waller, TX':'48473',
                'Galveston, TX':'48167','Harris, TX':'48201','Liberty, TX':'48291',
                'Chambers, TX':'48071','Brazoria, TX':'48039','Austin, TX':'48015',
                'Montgomery, TX':'48339'}
houmetro_cntys_swap = {v: k for k, v in houmetro_cntys.items()}
houmetro_fips_ints = [ int(x) for x in list(houmetro_cntys.values()) ]
houmetro_fips_strs = [ str(x) for x in list(houmetro_cntys.values()) ]
houmetro_nms = [ str(x) for x in list(houmetro_cntys.keys()) ]


tracts10_shp = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2010/TRACT/2010/tl_2010_48_tract10.zip')
tracts10_shp['cnty_geoid'] = tracts10_shp['STATEFP10'].astype(str) + tracts10_shp['COUNTYFP10'].astype(str)
tracts10_shp['cnty_nm'] = tracts10_shp['cnty_geoid'].map(houmetro_cntys_swap)
tracts10_shp.rename(columns={'GEOID10':'tract_geoid'},inplace=True)
houtracts10_shp = tracts10_shp.loc[tracts10_shp['cnty_geoid'].isin(houmetro_fips_strs)]
houtracts10_shp = houtracts10_shp[['cnty_geoid','cnty_nm','tract_geoid','geometry']]

tracts20_shp = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2024/TRACT/tl_2024_48_tract.zip')
tracts20_shp['cnty_geoid'] = tracts20_shp['STATEFP'].astype(str) + tracts20_shp['COUNTYFP'].astype(str)
tracts20_shp['cnty_nm'] = tracts20_shp['cnty_geoid'].map(houmetro_cntys_swap)
tracts20_shp.rename(columns={'GEOID':'tract_geoid'},inplace=True)
houtracts20_shp = tracts20_shp.loc[tracts20_shp['cnty_geoid'].isin(houmetro_fips_strs)]
houtracts20_shp = houtracts20_shp[['cnty_geoid','cnty_nm','tract_geoid','geometry']]

In [ ]:
print(len(houtracts10_shp))
print(houtracts10_shp.dtypes)
display(houtracts10_shp.head(1))

print(len(houtracts20_shp))
print(houtracts20_shp.dtypes)
display(houtracts20_shp.head(1))

In [3]:
########################################
# CDC asthma data
########################################

#load national data
cdc = pd.read_csv('../data/CDC/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2024_release_20250103.csv')

#restrict to just hou metro counties
hou_cdc = cdc.loc[cdc['CountyFIPS'].isin(houmetro_fips_ints)]

#delete cdc cause it's huge and we don't need it anymore
del cdc

#restrict to just asthma
hou_asthma = hou_cdc.loc[hou_cdc['MeasureId'] == 'CASTHMA']

#create correct type on tract_geoid
hou_asthma['tract_geoid'] = hou_asthma['LocationName'].astype(str)

#too many columns, reduce and rename
rename_cols = {'Data_Value':'dv','TotalPopulation':'p','TotalPop18plus':'py',
               'tract_geoid':'tract_geoid'}
hou_asthma.rename(columns=rename_cols,inplace=True)
hou_asthma = hou_asthma[list(rename_cols.values())]

#connect to 2020 tracts - GEOID
hou_asthma_geo = houtracts20_shp.merge(hou_asthma,on='tract_geoid',how='left')

#save
hou_asthma_geo_export = hou_asthma_geo.loc[~hou_asthma_geo['dv'].isna()]
hou_asthma_geo_export.to_file('../GIS/for-map/cdc-asthma-houmetro.geojson')

/var/folders/2g/mqj0d6v95vbfcqgf7kpgrsrc0000gr/T/ipykernel_90787/48751885.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hou_asthma['tract_geoid'] = hou_asthma['LocationName'].astype(str)
/var/folders/2g/mqj0d6v95vbfcqgf7kpgrsrc0000gr/T/ipykernel_90787/48751885.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hou_asthma.rename(columns=rename_cols,inplace=True)


In [5]:
#get max and min values for searchable map formatting
print(hou_asthma_geo.dv.min())
print(hou_asthma_geo.dv.max())

6.1
15.1


In [ ]:
#just making sure things look right
print(len(hou_asthma_geo))
print(hou_asthma_geo.columns)
display(hou_asthma_geo.head())

In [ ]:
#checking for missing valuse
print(len(hou_asthma_geo.loc[hou_asthma_geo['dv'].isna()]))
display(hou_asthma_geo.loc[hou_asthma_geo['dv'].isna()])

In [4]:
########################################
# CDC life expectancy data
########################################

#load national data
life_ex_us = pd.read_csv('../data/CDC/U.S._Life_Expectancy_at_Birth_by_State_and_Census_Tract_-_2010-2015.csv')

#restrict to just hou metro counties
life_ex_us['County'] = life_ex_us['County'].str.replace(' County','')
hou_life = life_ex_us.loc[(life_ex_us['State'] == 'Texas')&(life_ex_us['County'].isin(houmetro_nms))]

#delete us data cause we don't need it
del life_ex_us

#get rid of values that don't have tract numbers
hou_life = hou_life.loc[~hou_life['Census Tract Number'].isna()]

#creately format tract_geoid
hou_life['tract_fips'] = hou_life['Census Tract Number'].apply(lambda x: f'{x:.2f}')
hou_life['tract_fips'] = hou_life['tract_fips'].astype(str).str.replace('.','')
hou_life['cnty_fips'] = hou_life['County'].map(houmetro_cntys)
hou_life['tract_geoid'] = hou_life['cnty_fips'] + hou_life['tract_fips']

#too many columns, reduce and rename
rename_cols = {'Life Expectancy':'ex','Life Expectancy Range':'exr', 
               'Life Expectancy Standard Error':'exe','tract_geoid':'tract_geoid'}
hou_life.rename(columns=rename_cols,inplace=True)
hou_life = hou_life[list(rename_cols.values())]

#connect to 2010 tracts - GEOID
hou_life_geo = houtracts10_shp.merge(hou_life,on='tract_geoid')

#save
hou_life_geo_export = hou_life_geo.loc[~hou_life_geo['ex'].isna()]
hou_life_geo_export.to_file('../GIS/for-map/cdc-lifeexpectancy-houmetro.geojson')

In [6]:
#get max and min values for searchable map formatting
print(hou_life_geo.ex.min())
print(hou_life_geo.ex.max())

65.7
89.1


In [ ]:
#just making sure they all join
print('hou_life:',len(hou_life))
print('hou_life_geo:',len(hou_life_geo))
print(hou_life.columns)
display(hou_life.loc[~hou_life['tract_geoid'].isin(list(hou_life_geo['tract_geoid'].unique()))].head())

In [7]:
########################################
# Poverty data
########################################

#load national data
pov = pd.read_csv('../data/ACSST5Y2023.S1901_2025-01-03T110020/ACSST5Y2023.S1901-Data.csv')

#drop the first row that's the hr column names
pov = pov.iloc[1:]

#create correct type on fields
pov['tract_geoid'] = pov['GEO_ID'].str.replace('1400000US','')
pov['cnty_geoid'] = pov['tract_geoid'].str[:5]
pov['S1901_C01_012E'] = pd.to_numeric(pov['S1901_C01_012E'],errors='coerce')
pov['S1901_C01_012M'] = pd.to_numeric(pov['S1901_C01_012M'],errors='coerce')

#restrict to just hou metro counties
hou_pov = pov.loc[pov['cnty_geoid'].isin(houmetro_fips_strs)]

#delete cdc cause it's huge and we don't need it anymore
del pov

#too many columns, reduce and rename
keep_cols = {'tract_geoid':'tract_geoid','S1901_C01_001E':'hh',
             'S1901_C01_012E':'mi','S1901_C01_012M':'mimoe'}
hou_pov = hou_pov[list(keep_cols.keys())]
hou_pov.rename(columns=keep_cols,inplace=True)

#connect to 2020 tracts - GEOID
hou_pov_geo = houtracts20_shp.merge(hou_pov,on='tract_geoid',how='left')

#save
hou_pov_geo_export = hou_pov_geo.loc[~hou_pov_geo['mi'].isna()]
hou_pov_geo.to_file('../GIS/for-map/census-hh-income-houmetro.geojson')

/var/folders/2g/mqj0d6v95vbfcqgf7kpgrsrc0000gr/T/ipykernel_90787/1756456637.py:6: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  pov = pd.read_csv('../data/ACSST5Y2023.S1901_2025-01-03T110020/ACSST5Y2023.S1901-Data.csv')


In [8]:
#get max and min values for searchable map formatting
print(hou_pov_geo.mi.min())
print(hou_pov_geo.mi.max())

11592.0
246705.0


In [ ]:
print(len(hou_pov))
print(len(hou_pov_geo))

In [10]:
#just checking how many have outrageous MOE
hou_pov['moe_share'] = (hou_pov['mimoe']/hou_pov['mi'])*100

#share with moe above 30%
print('Share with MOE over 30%:',(len(hou_pov.loc[hou_pov['moe_share']>29.99])/len(hou_pov))*100)

display(hou_pov.sort_values('moe_share',ascending=False).head(20))
display(hou_pov.sort_values('moe_share',ascending=True).head(20))

Share with MOE over 30%: 33.68421052631579


,tract_geoid,hh,mi,mimoe,moe_share
2515,48157672802,916,75385.0,72712.0,96.454202
5100,48339694305,2096,77500.0,74499.0,96.127742
3860,48201550304,877,34306.0,32694.0,95.301114
3093,48201252602,1159,43734.0,41496.0,94.882700
2926,48201221701,1433,42078.0,38403.0,91.266220
3339,48201350201,1123,74619.0,66241.0,88.772297
3603,48201452703,1215,48893.0,43099.0,88.149633
2694,48167724000,380,38000.0,33131.0,87.186842
3912,48201552604,1157,73967.0,64307.0,86.940122
2931,48201222100,1287,41759.0,36040.0,86.304749


,tract_geoid,hh,mi,mimoe,moe_share
3987,48201555600,2243,141665.0,2448.0,1.728020
4995,48339690408,1756,109024.0,2796.0,2.564573
2649,48167721206,1431,244952.0,7971.0,3.254107
5110,48339694503,4247,138157.0,4766.0,3.449699
3658,48201510601,1996,125652.0,4415.0,3.513673
3781,48201540502,1615,92772.0,3609.0,3.890182
3291,48201340302,2500,144127.0,5813.0,4.033248
3883,48201551300,1122,138200.0,5585.0,4.041245
4671,48291700500,1004,95920.0,4400.0,4.587156
3728,48201531100,1329,176091.0,8497.0,4.825346
